## Dividend Yield

### Dividend Yield (%) = Soma dos proventos pagos / preço da ação

#### Bibliotecas

In [1]:
import yfinance as yf                     # biblioteca para obtenção de dados de Investimento
import requests                           # biblioteca para fazer requisições na Internet
import pandas as pd                       # biblioteca Pandas para manipulação de dados
import numpy as np                        # biblioteca Numpy para manipulação de dados numéricos
pd.set_option('mode.chained_assignment', None)
from datetime import date                 # funções para manipulação de datas

#### Definindo Função

In [2]:
def dividend_yield(codigo):
    # --------------------------------------------
    # >> PASSO 1: OBTER O PREÇO DA AÇÃO
    # --------------------------------------------
    preco = yf.Ticker(codigo + '.SA').history(period='1d')['Close'].values[0].round(2)
    # --------------------------------------------
    # >> PASSO 2: BAIXAR OS DADOS DO SITE FUNDAMENTUS
    # --------------------------------------------
    url = 'https://www.fundamentus.com.br/proventos.php?papel=' + codigo + '&tipo=2'
    cabecalho = {              #informações para fingir ser um navegador
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebkit/537.36 (KHTML, like Gecko) \
        Chrome/50.0.2661.75 Safari/537.36",
        "x-Requested-With": "XMLHttpRequest" 
    }
    r = requests.get(url, headers=cabecalho)   # cria uma requisição com a URL e o cabeçalho
    dados_fundamentus = pd.read_html(r.text, decimal=',', thousands='.')[0] # faz a busca pelos dados na URL
    # --------------------------------------------
    # >> PASSO 3: MANIPULAÇÃO DOS DADOS
    # --------------------------------------------
    dados_fundamentus['Data'] = pd.to_datetime(dados_fundamentus['Data'], dayfirst=True)  # converte a data pro tipo datetime
    proventos = dados_fundamentus[['Data', 'Valor', 'Tipo']]  # cria um novo DataFrame com as colunas que interessam
    proventos['Valor Líquido'] = np.where(proventos['Tipo']=='DIVIDENDO', proventos['Valor'], proventos['Valor']*0.85)
    # --------------------------------------------
    # >> PASSO 4: OBTER A SOMA DOS PROVENTOS DOS ÚLTIMOS 12 MESES
    # --------------------------------------------
    data_hoje = date.today()   # obtém a data atual
    data_limite = f'{data_hoje.year-1}-{data_hoje.month}-{data_hoje.day}' # obtém uma string com a data de 12 meses atrás     
    proventos = proventos[proventos['Data'] > data_limite]    # filtra os dados dos últimos 12 meses
    proventos_12_meses = proventos['Valor Líquido'].sum()     # soma os proventos líquidos pagos
    # --------------------------------------------
    # >> PASSO 5: RETORNA O DIVIDEND YIELD
    # --------------------------------------------
    return(proventos_12_meses/preco)*100.0

In [3]:
dividend_yield('BBAS3')

8.390904232578023